<H1>Manage Librairies and IDs</h1>

In [1]:
# The code was removed by Watson Studio for sharing.

In [2]:
LIMIT = 100

In [3]:
#Import Librairies
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import pandas as pd
import numpy as np
import random as random
from sklearn.cluster import KMeans 
import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 
!pip install folium
import folium

<H1> Functions </H1>

In [4]:
#Get list of categories
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        print (categories_list)
        return categories_list[0]['name']

In [5]:
#function to get the nearby venues from Latitude, Longitude, Nb de venues max, and radius max.
def GetVenuesNearByInDataFrame(Lat_param,Lng_param,Nb_max_venues,radius):
    url ='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&limit={}&radius={}'.format(
    CLIENT_ID,CLIENT_SECRET, VERSION, Lat_param, Lng_param ,Nb_max_venues ,radius)
    results = requests.get(url).json()
    print(results)
    venues = results['response']['groups'][0]['items'] 
    nearby_venues = pd.json_normalize(venues) # flatten JSON
    # filter columns
    filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng','venue.location.city','venue.location.postalCode']
    nearby_venues =nearby_venues.loc[:, filtered_columns]
    # filter the category for each row
    nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
    # clean columns
    nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
    nearby_venues.head()
    
    return (nearby_venues )

In [6]:
#Get the data for Montmatre, a parisian neighborhood
MontmartreLat=48.8857636427167
MontmartreLng=2.3349730916262725
DataFromParis = GetVenuesNearByInDataFrame(MontmartreLat,MontmartreLng,50,300)

{'meta': {'code': 200, 'requestId': '5fa5ad4d442f6c14d080bddc'}, 'response': {'suggestedFilters': {'header': 'Tap to show:', 'filters': [{'name': 'Open now', 'key': 'openNow'}]}, 'headerLocation': 'Clignancourt', 'headerFullLocation': 'Clignancourt, Paris', 'headerLocationGranularity': 'neighborhood', 'totalResults': 70, 'suggestedBounds': {'ne': {'lat': 48.8884636454167, 'lng': 2.3390715030046434}, 'sw': {'lat': 48.883063640016694, 'lng': 2.3308746802479017}}, 'groups': [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4d541304d4a6721e9b5691ac', 'name': 'Boulangerie Alexine', 'location': {'address': '40 rue Lepic', 'lat': 48.88614133131494, 'lng': 2.3344767093658447, 'labeledLatLngs': [{'label': 'display', 'lat': 48.88614133131494, 'lng': 2.3344767093658447}], 'distance': 55, 'postalCode': '75018', 'cc': 'FR', 'city': 'Pa

In [7]:
DataFromParis.shape

(50, 6)

In [8]:
SydneyLat= -33.83083506008578
SydneyLng= 151.2199836084402
DataFromSydney = GetVenuesNearByInDataFrame(SydneyLat,SydneyLng,100,3000)
DataFromSydney

{'meta': {'code': 200, 'requestId': '5fa5ad4da97868185a345875'}, 'response': {'suggestedFilters': {'header': 'Tap to show:', 'filters': [{'name': 'Open now', 'key': 'openNow'}]}, 'headerLocation': 'Sydney', 'headerFullLocation': 'Sydney', 'headerLocationGranularity': 'city', 'totalResults': 180, 'suggestedBounds': {'ne': {'lat': -33.803835033085754, 'lng': 151.2524262503318}, 'sw': {'lat': -33.85783508708581, 'lng': 151.1875409665486}}, 'groups': [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4cd8712815d8b60c14fc650e', 'name': 'Essence Coffee', 'location': {'address': 'Shop 3, 148 Wycombe Rd', 'lat': -33.83190296551829, 'lng': 151.22235012479607, 'labeledLatLngs': [{'label': 'display', 'lat': -33.83190296551829, 'lng': 151.22235012479607}], 'distance': 249, 'postalCode': '2089', 'cc': 'AU', 'city': 'Neutral Bay', 'state

,name,categories,lat,lng,city,postalCode
0,Essence Coffee,Café,-33.831903,151.222350,Neutral Bay,2089
1,Jugemu & Shimbashi,Japanese Restaurant,-33.830956,151.224333,Neutral Bay,2089
2,Sam Thai,Thai Restaurant,-33.831275,151.221663,Neutral Bay,2089
3,Tobikiri Japanese Kitchen,Japanese Restaurant,-33.832200,151.222560,Neutral Bay,NaN
4,Sunset Cinema,Movie Theater,-33.831450,151.209150,Sydney,NaN
...,...,...,...,...,...,...
95,Taronga Zoo Bird Show,Zoo Exhibit,-33.843986,151.239308,Mosman,2088
96,Celsius Coffee Co.,Café,-33.849403,151.220256,Kirribilli,2061
97,Park Hyatt Sydney,Hotel,-33.856023,151.209225,Sydney,2000
98,Guzman Y Gomez,Mexican Restaurant,-33.824645,151.201232,Crows Nest,2065


In [9]:
#Concatenate the venus from the 2 cities
df_venues=pd.concat ([DataFromParis,DataFromSydney]).reset_index()
df_venues.drop(['index'],axis=1,inplace =True)
df_venues.head()

,name,categories,lat,lng,city,postalCode
0,Boulangerie Alexine,Bakery,48.886141,2.334477,Paris,75018
1,Guilo Guilo,Japanese Restaurant,48.885942,2.337048,Paris,75018
2,Terrass'' Hotel,Hotel,48.886532,2.333159,Paris,75018
3,Al Caratello,Italian Restaurant,48.885248,2.336002,Paris,75018
4,Le Grenier à Pain,Bakery,48.885283,2.336718,Paris,75018


<H1>Prepare the dataset</H1>

In [10]:
#turn the categorical data to a numeric (Hot encoding)
Data_hot = pd.get_dummies (df_venues[['categories']],prefix='',prefix_sep='')
Data_hot

# add neighborhood column back to dataframe
Data_hot['name'] = df_venues['name'] 
Data_hot['lat'] = df_venues['lat'] 
Data_hot['lng'] = df_venues['lng'] 
Data_hot['city'] = df_venues['city'] 
Data_hot['postalCode'] = df_venues['postalCode'] 


#move the fixed columns to the font.
fixed_columns = [Data_hot.columns[-1]] + list(Data_hot.columns[:-1])
Data_hot = Data_hot[fixed_columns]
fixed_columns = [Data_hot.columns[-1]] + list(Data_hot.columns[:-1])
Data_hot = Data_hot[fixed_columns]
fixed_columns = [Data_hot.columns[-1]] + list(Data_hot.columns[:-1])
Data_hot = Data_hot[fixed_columns]
fixed_columns = [Data_hot.columns[-1]] + list(Data_hot.columns[:-1])
Data_hot = Data_hot[fixed_columns]
fixed_columns = [Data_hot.columns[-1]] + list(Data_hot.columns[:-1])
Data_hot = Data_hot[fixed_columns]
Data_hot.head()

,name,lat,lng,city,postalCode,American Restaurant,Bakery,Bar,Beach,Beer Bar,...,Street Art,Tennis Court,Thai Restaurant,Theater,Trail,Turkish Restaurant,Wine Bar,Wine Shop,Zoo,Zoo Exhibit
0,Boulangerie Alexine,48.886141,2.334477,Paris,75018,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Guilo Guilo,48.885942,2.337048,Paris,75018,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Terrass'' Hotel,48.886532,2.333159,Paris,75018,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Al Caratello,48.885248,2.336002,Paris,75018,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Le Grenier à Pain,48.885283,2.336718,Paris,75018,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
#Group the data by PostalCode
city_grouped = Data_hot.groupby(['postalCode']).mean().reset_index()
city_grouped.head()

,postalCode,lat,lng,American Restaurant,Bakery,Bar,Beach,Beer Bar,Bistro,Bridge,...,Street Art,Tennis Court,Thai Restaurant,Theater,Trail,Turkish Restaurant,Wine Bar,Wine Shop,Zoo,Zoo Exhibit
0,2000,-33.855800,151.212677,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.142857,...,0.0,0.0,0.000000,0.285714,0.0,0.0,0.0,0.0,0.0,0.0
1,2055,-33.838870,151.207358,0.000000,1.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,2060,-33.838676,151.206868,0.047619,0.0,0.095238,0.0,0.0,0.047619,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,2061,-33.849151,151.215229,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,2062,-33.822722,151.212914,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.333333,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


<H1>Apply K-means to cluster the Areas according their most common venues</H1>

In [12]:
X = city_grouped.values[:,4:]

In [13]:
k=4
k_means = KMeans(init="k-means++", n_clusters=k, n_init=21)
k_means.fit(X)
labels = k_means.labels_
k_means

KMeans(n_clusters=4, n_init=21)

In [14]:
labels
city_grouped["Label"] = labels
city_grouped.head()

,postalCode,lat,lng,American Restaurant,Bakery,Bar,Beach,Beer Bar,Bistro,Bridge,...,Tennis Court,Thai Restaurant,Theater,Trail,Turkish Restaurant,Wine Bar,Wine Shop,Zoo,Zoo Exhibit,Label
0,2000,-33.855800,151.212677,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.142857,...,0.0,0.000000,0.285714,0.0,0.0,0.0,0.0,0.0,0.0,0
1,2055,-33.838870,151.207358,0.000000,1.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,3
2,2060,-33.838676,151.206868,0.047619,0.0,0.095238,0.0,0.0,0.047619,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1
3,2061,-33.849151,151.215229,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1
4,2062,-33.822722,151.212914,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.333333,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0


In [15]:
#Get the color for each cluster.
city_grouped.loc[city_grouped['Label'] == 0, 'color'] = '#000000'
city_grouped.loc[city_grouped['Label'] == 1, 'color'] = '#ff0000'
city_grouped.loc[city_grouped['Label'] == 2, 'color'] = '#ff8000'
city_grouped.loc[city_grouped['Label'] == 3, 'color'] = '#ff8080'
#city_grouped.loc[city_grouped['Label'] == 4, 'color'] = '#FF6F61' 
#city_grouped.loc[city_grouped['Label'] == 5, 'color'] = '#6B5B95' 
#city_grouped.loc[city_grouped['Label'] == 6, 'color'] = '#92A8D1' 
#city_grouped.loc[city_grouped['Label'] == 7, 'color'] = '#B565A7' 
#city_grouped.loc[city_grouped['Label'] == 8, 'color'] = '#009B77' 
#city_grouped.loc[city_grouped['Label'] == 9, 'color'] = '#DD4124' 
#city_grouped.loc[city_grouped['Label'] == 10, 'color'] = '#45B8AC' 
city_grouped.head ()

,postalCode,lat,lng,American Restaurant,Bakery,Bar,Beach,Beer Bar,Bistro,Bridge,...,Thai Restaurant,Theater,Trail,Turkish Restaurant,Wine Bar,Wine Shop,Zoo,Zoo Exhibit,Label,color
0,2000,-33.855800,151.212677,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.142857,...,0.000000,0.285714,0.0,0.0,0.0,0.0,0.0,0.0,0,#000000
1,2055,-33.838870,151.207358,0.000000,1.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,3,#ff8080
2,2060,-33.838676,151.206868,0.047619,0.0,0.095238,0.0,0.0,0.047619,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1,#ff0000
3,2061,-33.849151,151.215229,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1,#ff0000
4,2062,-33.822722,151.212914,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.333333,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0,#000000


In [16]:
#Get the Label/Cluster number from paris. It will be used to filter Sydney dataset
MontmatreCluster = city_grouped.loc[city_grouped['postalCode'] == '75018']['Label'].values
MontmatreCluster

array([0], dtype=int32)

<H1>Plug the data on a graph.</h1>

<h3>Get the data for Sydney like Paris areas.
Generate 1 maps for all Sydney areas, 1 map for only the parisian like areas.
For parisian like areas, display all the venues splited by categories</h3>

In [17]:
#Get the Sydney Neighborhood similar to Montmatre  (Sydney postal codes are 4 digits. Paris = 5 digits)
SydneyAreasLikeParis = city_grouped.loc[(city_grouped['postalCode'].astype('int64') <=10000) & (city_grouped['Label'] ==MontmatreCluster[0])  ]
AllSydney = city_grouped.loc[(city_grouped['postalCode'].astype('int64') <=10000) ]

SydneyAreasLikeParis

,postalCode,lat,lng,American Restaurant,Bakery,Bar,Beach,Beer Bar,Bistro,Bridge,...,Thai Restaurant,Theater,Trail,Turkish Restaurant,Wine Bar,Wine Shop,Zoo,Zoo Exhibit,Label,color
0,2000,-33.855800,151.212677,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,...,0.000000,0.285714,0.00,0.0,0.0,0.0,0.0,0.0,0,#000000
4,2062,-33.822722,151.212914,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.333333,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0,#000000
9,2090,-33.829859,151.227430,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.25,0.0,0.0,0.0,0.0,0.0,0,#000000


<h2>Map for all Sydney areas</h2>

In [28]:
#Generate the map for all Sydney
sydney_area_map = folium.Map(location=[ -33.87285279645642,151.2085876435312],zoom_start =12)
area = folium.map.FeatureGroup()
for lat, lng,color in zip(AllSydney.lat, AllSydney.lng,AllSydney.color):
    area.add_child(
        folium.CircleMarker(
            [lat, lng],
            color=color,
            fill=True,
            fill_color='color',
            fill_opacity=0.6,       
        )
    )
sydney_area_map.add_child(area)
# display map
sydney_area_map

<h2>Map for Sydney like Montmatre areas (same cluster)</h2>

In [26]:
#Generate the map for Sydney neighborhood like the parisian one
sydney_like_area_map = folium.Map(location=[ -33.82285279645642,151.2085876435312],zoom_start =12)
area = folium.map.FeatureGroup()
for lat, lng,color in zip(SydneyAreasLikeParis.lat, SydneyAreasLikeParis.lng,SydneyAreasLikeParis.color):
    area.add_child(
        folium.CircleMarker(
            [lat, lng],
            color=color,
            fill=True,
            fill_color='color',
            fill_opacity=0.6,       
        )
    )
sydney_like_area_map.add_child(area)
# display map
sydney_like_area_map

<H1>Display all the venues for the Sydney areas like Montmatre</H1>

In [20]:
#Merge the venues and the Areas dataframe in order to be able to filter the venues based on the postal code
VenuesSydney= SydneyAreasLikeParis[['postalCode']].merge(df_venues, how='left', left_on='postalCode', right_on='postalCode')

<H3>Get randomly a color 1 color for each categorie </h3>

In [21]:
#Get the unique list of the categories
#countCat = VenuesSydney['categories'].nunique()
List_Categories=VenuesSydney.categories.unique()
List_Categories=pd.DataFrame(List_Categories,columns=['categories'])

#Get random color for each venue categorie
List_Categories['color']=0
for cat in List_Categories['categories'] :
    r = lambda: random.randint(0,255)
    List_Categories.loc[List_Categories['categories'] == cat, ['color']] = ('#%02X%02X%02X' % (r(),r(),r()))
List_Categories

,categories,color
0,Bridge,#E8C853
1,Concert Hall,#1B39F7
2,Scenic Lookout,#199498
3,Theater,#DD2F09
4,Opera House,#45595C
5,Hotel,#88DE58
6,Thai Restaurant,#F09BFB
7,Japanese Restaurant,#C4D815
8,Fruit & Vegetable Store,#372A49
9,Trail,#6B4391


In [22]:
#Merge venues and color mapping - Get the color in the dataframe
VenuesSydney= VenuesSydney.merge(List_Categories, how='left', left_on='categories', right_on='categories')
VenuesSydney.head()

,postalCode,name,categories,lat,lng,city,color
0,2000,Sydney Harbour Bridge,Bridge,-33.852289,151.210681,Sydney,#E8C853
1,2000,Sydney Opera House - Concert Hall,Concert Hall,-33.856595,151.215058,Circular Quay,#1B39F7
2,2000,Harbour Bridge Pylon Lookout,Scenic Lookout,-33.854580,151.209492,Sydney,#199498
3,2000,Sydney Opera House - Studio,Theater,-33.856991,151.214482,Sydney,#DD2F09
4,2000,Sydney Opera House,Opera House,-33.857260,151.215040,Sydney,#45595C


<h2>Map for Sydney like Montmatre areas</h2>

In [27]:
sydney_venues_map = folium.Map(location=[-33.87083506008578, 151.2199836084402],zoom_start =13)
venues = folium.map.FeatureGroup()

for lat, lng,name,color,cat in zip(VenuesSydney.lat, VenuesSydney.lng,VenuesSydney.name,VenuesSydney.color,VenuesSydney.categories):
    venues.add_child(
        folium.CircleMarker(
            [lat, lng],
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.6,
            tooltip=name +' - '+cat
        )
    )
sydney_venues_map.add_child(venues)
# display map
sydney_venues_map